# Python'da Görüntü İşlemeye Giriş
Python'da görüntü işleme işlemleri sıklıkla kullanılan iki modül:
* pillow
* OpenCV

## Pillow
Python Imaging Library’nin geliştirilmiş versiyonudur. Basit görüntü işlemleri (açma, kaydetme, boyutlandırma, filtreleme) için kullanılır.
## OpenCV
 Görüntü işleme ve bilgisayarla görme (computer vision) uygulamaları için güçlü bir kütüphanedir. Gerçek zamanlı görüntü işleme, yüz tanıma, video analizi gibi alanlarda da kullanılır.

In [ ]:
pip install pillow opencv-python

## Bir görüntüyü açma

In [ ]:
from PIL import Image

# Görüntü aç
img = Image.open("resim.jpg")

# Görüntü bilgisi
print(img.format, img.size, img.mode)

# Görüntüyü göster
img.show()

## Görsel boyutlandırma

In [ ]:
resized = img.resize((200, 200))
resized.show()

## Gri Tonlamaya Çevirme

In [ ]:
# Resmi gri tonlamaya çevir
gray = img.convert("L")
gray.show()

## Görüntüyü kaydetme

In [ ]:
gray.save("gri_pillow.jpg",format="JPEG", quality=10)

In [ ]:
from PIL import Image, ImageFilter, ImageEnhance, ImageDraw

# Görüntüyü döndürme
rotated = img.rotate(45)  # 45 derece döndür
rotated.show()


In [ ]:
# Görüntüyü kırpma (crop)
width, height = img.size
# Sol üstten bir bölümü kes (left, upper, right, lower)
cropped = img.crop((0, 0, width//2, height//2))
cropped.show()

In [ ]:
# Filtre uygulama
blurred = img.filter(ImageFilter.BLUR)
edges = img.filter(ImageFilter.FIND_EDGES)
blurred.show()
edges.show()

In [ ]:
# Parlaklık ayarlama
enhancer = ImageEnhance.Brightness(img)
brightened = enhancer.enhance(1.7)  # Parlaklığı %70 artır
brightened.show()


In [ ]:
# Kontrast ayarlama
contrast = ImageEnhance.Contrast(img)
high_contrast = contrast.enhance(1.7)  # Kontrastı %70 artır
high_contrast.show()

In [ ]:
# Görüntü üzerine çizim yapma
draw_img = img.copy()
draw = ImageDraw.Draw(draw_img)
draw.rectangle((50, 50, 200, 200), outline="red")
draw.text((100, 100), "Merhaba!", fill="white")
draw_img.show()


In [ ]:
# Görüntüyü yatay ve dikey çevirme (flip)
flipped_horizontal = img.transpose(Image.FLIP_LEFT_RIGHT)
flipped_vertical = img.transpose(Image.FLIP_TOP_BOTTOM)
flipped_horizontal.show()
flipped_vertical.show()



In [ ]:
# İki görüntüyü birleştirme
width, height = img.size
combined = Image.new('RGB', (width * 2, height))
combined.paste(img, (0, 0))
combined.paste(gray.convert('RGB'), (width, 0))
combined.show()

In [ ]:
# Thumbnail oluşturma (oranı koruyarak küçültme)
thumbnail = img.copy()
thumbnail.thumbnail((100, 100))  # En boy oranını koruyarak 100x100'den küçük yapar
thumbnail.show()

In [ ]:
# Renk kanallarına erişim
r, g, b = img.split()
# Sadece kırmızı kanalı olan görüntü
red_channel = Image.merge("RGB", (r, g.point(lambda i: 0), b.point(lambda i: 0)))
red_channel.show()

In [ ]:
from PIL import ImageOps

# Grayscale görüntünün renklerini tersine çevirme (inversion)
# Burada b kanalını aldığımda gri tonlamalı görüntü elde ediyorum
b.show()
inverted = ImageOps.invert(b)
inverted.show()

# Histogram eşitleme ile görüntü kontrastını iyileştirme
equalized = ImageOps.equalize(b)
equalized.show()

# Otomatik kontrast ayarı ile "img" görüntüsünü düzenleme
auto_contrast = ImageOps.autocontrast(img)
auto_contrast.show()

In [ ]:
def apply_sepia(input_image):
    sepia = input_image.copy()
    sepia = sepia.convert("RGB")
    width, height = sepia.size
    pixels = sepia.load()
    
    for y in range(height):
        for x in range(width):
            r, g, b = sepia.getpixel((x, y))
            tr = int(0.393 * r + 0.769 * g + 0.189 * b)
            tg = int(0.349 * r + 0.686 * g + 0.168 * b)
            tb = int(0.272 * r + 0.534 * g + 0.131 * b)
            pixels[x, y] = (min(tr, 255), min(tg, 255), min(tb, 255))
    
    return sepia

sepia_image = apply_sepia(img)
sepia_image.show()

In [ ]:
# Y,Cb,Cr renk uzayına dönüştür ve kanalları tek tek göster
ycbcr = img.convert("YCbCr")
y, cb, cr = ycbcr.split()
y.show()
cb.show()
cr.show()


In [ ]:
# --- YCbCr Kanallarının Görselleştirilmesi---

ycbcr = img.convert("YCbCr")
y, cb, cr = ycbcr.split()


# Y is luminance, its most direct representation is grayscale.
y.show(title="Y Channel (Luminance)")

print("Displaying Cb channel (Blue Difference - Color Visualization)")
# Create constant channels for Y (mid-gray) and Cr (neutral chroma)
# 128 is the midpoint for 8-bit Y and the neutral value for Cb/Cr
mid_luma = Image.new('L', cb.size, 128)
neutral_cr = Image.new('L', cb.size, 128)

# Merge channels: Y=constant(128), Cb=variable, Cr=constant(128)
# This shows how Cb varies from neutral blue/yellow at mid-luminance
cb_color_vis_ycbcr = Image.merge('YCbCr', (mid_luma, cb, neutral_cr))

# Convert back to RGB for display
cb_color_vis_rgb = cb_color_vis_ycbcr.convert('RGB')
cb_color_vis_rgb.show(title="Cb Visualization (Y=128, Cr=128)")



print("Displaying Cr channel (Red Difference - Color Visualization)")
# Create constant channel for Cb (neutral chroma)
# We can reuse mid_luma from before
neutral_cb = Image.new('L', cr.size, 128)

# Merge channels: Y=constant(128), Cb=constant(128), Cr=variable
# This shows how Cr varies from neutral red/cyan at mid-luminance
cr_color_vis_ycbcr = Image.merge('YCbCr', (mid_luma, neutral_cb, cr))

# Convert back to RGB for display
cr_color_vis_rgb = cr_color_vis_ycbcr.convert('RGB')
cr_color_vis_rgb.show(title="Cr Visualization (Y=128, Cb=128)")

print("\nExplanation:")
print(" - Y Channel: Shown as grayscale, representing the image's luminance (brightness).")
print(" - Cb Visualization: Shows the blue-difference component. Colors will range from blueish (low Cb) to yellowish (high Cb) around a neutral gray.")
print(" - Cr Visualization: Shows the red-difference component. Colors will range from reddish (high Cr) to cyanish/greenish (low Cr) around a neutral gray.")

## OpenCV Kullanımı

In [ ]:
import cv2

# Görüntü aç
img_cv = cv2.imread("resim.jpg")

# Görüntüyü göster
cv2.imshow("Goruntu", img_cv)

# Pencereyi kapatmak için bir tuşa basılmasını bekle
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
resized_cv = cv2.resize(img_cv, (200, 200))
cv2.imshow("Yeniden Boyutlandirilmis", resized_cv)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# OpenCV örnekleri

# 1. img_cv görüntüsünü gri tonu formatına çevirelim.
gray_cv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
cv2.imshow("Grayscale Image", gray_cv)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 2. Gaussian Blur filtresi uygulayalım.
blurred_cv = cv2.GaussianBlur(img_cv, (7, 7), 0)
cv2.imshow("Gaussian Blurred Image", blurred_cv)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 3. Canny kenar tespiti ile kenarları belirleyelim.
edges_cv = cv2.Canny(img_cv, 100, 200)
cv2.imshow("Canny Kenarları", edges_cv)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 4. resized_cv görüntüsüne bir metin ekleyelim.
annotated_cv = resized_cv.copy()
cv2.putText(annotated_cv, "Merhaba OpenCV", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
cv2.imshow("Yazılı Image", annotated_cv)
cv2.waitKey(0)
cv2.destroyAllWindows()